In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
#!/bin/bash
! kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 1.84GB/s]


In [6]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('/content/IMDB Dataset.csv')

In [9]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
df.shape

(50000, 2)

In [11]:
df.dropna(inplace = True)

In [12]:
import re
import nltk
import string
! pip install contractions


import contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 14.1 MB/s eta 0:00:00


In [13]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [14]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [15]:
stop_word = stopwords.words('english')

lemma = WordNetLemmatizer()

In [16]:
def clean_text(text):
  text = contractions.fix(text)
  text = text.lower()
  text = re.sub(r'http\S+' , '' , text)
  text = re.sub(r'<.*?>' , '' , text)
  text = text.translate(str.maketrans('' , '' , string.punctuation))
  token = word_tokenize(text)
  final = [lemma.lemmatize(word) for word in token if word not in stop_word]
  return ' '.join(final)

In [17]:
df['review'] = df['review'].apply(clean_text)

In [18]:
df.head()

,review,sentiment
0,one reviewer mentioned watching 1 oz episode h...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake think zombie ...,negative
4,petter matteis love time money visually stunni...,positive


In [19]:
df['sentiment'] = df['sentiment'].map({'positive' :1 , 'negative' :0 })

In [20]:
x = df['review']
y = df['sentiment']

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
x_train , x_test, y_train , y_test = train_test_split(x ,y , train_size = 0.2 , random_state = 42)

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
token = Tokenizer(num_words = 10000 , oov_token = 'OOV')

In [25]:
token.fit_on_texts(x_train)

In [26]:
train_seq = token.texts_to_sequences(x_train)

In [27]:
len(train_seq)

10000

In [28]:
test_seq = token.texts_to_sequences(x_test)

In [29]:
max_len = max([len(seq) for seq in train_seq])
max_len

879

In [30]:
train_padding = pad_sequences(train_seq , maxlen = max_len , padding = 'post')

In [31]:
test_padding = pad_sequences(test_seq , maxlen = max_len , padding = 'post')

In [32]:
from keras.layers import *
from keras.models import *

In [33]:
input = Input(shape =( max_len,))

emb = Embedding(input_dim = 10000, output_dim = 32 , mask_zero = True)(input)

l1 = Bidirectional(LSTM(64 , activation = 'tanh', return_sequences = False))(emb)

l2 = Dropout(0.3)(l1)

o1 = Dense(1 , activation = 'sigmoid')(l2)

model = Model(inputs = input ,  outputs = o1)

In [34]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 879)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 879, 32)   │    320,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 879)       │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 128)       │     49,664 │ embedding[0][0],  │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │        129 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 369,793 (1.41 MB)

 Trainable params: 369,793 (1.41 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
from keras.optimizers import *
from keras.callbacks import *
from keras.losses import *

In [36]:
model.compile(optimizer = Adam(1e-3) , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [37]:
model.fit(train_padding , y_train , validation_data = (test_padding , y_test) , epochs  = 4 , verbose = 1 , batch_size =64 )

Epoch 1/4
157/157 ━━━━━━━━━━━━━━━━━━━━ 19s 85ms/step - accuracy: 0.6195 - loss: 0.6286 - val_accuracy: 0.7850 - val_loss: 0.5039
Epoch 2/4
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 81ms/step - accuracy: 0.8785 - loss: 0.3410 - val_accuracy: 0.8656 - val_loss: 0.3223
Epoch 3/4
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 80ms/step - accuracy: 0.9416 - loss: 0.1759 - val_accuracy: 0.8622 - val_loss: 0.3458
Epoch 4/4
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 82ms/step - accuracy: 0.9684 - loss: 0.1056 - val_accuracy: 0.8529 - val_loss: 0.3679


In [38]:
model.evaluate(test_padding , y_test)

1250/1250 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.8539 - loss: 0.3658


[0.3678847551345825, 0.8529250025749207]

In [39]:
model.evaluate(train_padding , y_train)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9767 - loss: 0.1042


[0.09615840762853622, 0.9789999723434448]

In [41]:
# 1️⃣ Save model
model.save("day_9_imdb_lstm.keras")

In [42]:
import joblib

joblib.dump(token , 'tokenizer.pkl')


joblib.dump(max_len , 'max_len.pkl')



['max_len.pkl']